In [1]:
import pickle
import numpy as np
import pandas as pd

In [11]:
movies_with_tag = pd.read_csv('data/all_movies.csv', lineterminator='\n')
ratings = pd.read_csv('data/ratings_new.csv')

In [35]:
movies_with_movie_index = pd.read_csv('data/all_movies-movie-index.csv', lineterminator='\n')
movies_with_movie_index.head()

,primaryTitle,Directors,Writers,tmdb_id,popularity,company_name,tmdb_avgRating,release_year_int,title,genres,document
0,Hollow Man,Paul Verhoeven,Gary Scott Thompson Andrew W. Marlowe Ron Vign...,9383,20.352,Columbia Pictures Red Wagon Entertainment Glob...,5.9,2000,Hollow Man (2000),Horror Sci-Fi Thriller,Paul Verhoeven Rhona Mitra Elisabeth Shue Kevi...
1,Nurse Betty,Neil LaBute None,James Flamberg John C. Richards Neil LaBute,10480,12.417,IMF Internationale Medien und Film GmbH & Co. ...,6.0,2000,Nurse Betty (2000),Comedy Crime Drama Romance Thriller,Neil LaBute None Morgan Freeman Renée Zellwege...
2,Supernova,Walter Hill,William Malone David C. Wilson Daniel Chuba Sy...,10384,13.120,Hammerhead Productions Screenland Pictures Uni...,5.0,2000,Supernova (2000),Adventure Sci-Fi Thriller,Walter Hill James Spader Robin Tunney Angela B...
3,Boiler Room,Ben Younger None,Ben Younger,14181,10.375,New Line Cinema,6.7,2000,Boiler Room (2000),Crime Drama Thriller,Ben Younger None Ben Affleck Vin Diesel Giovan...
4,Pitch Black,David Twohy None,Ken Wheat Jim Wheat David Twohy Jason Brubaker...,2787,16.980,Interscope Communications,6.8,2000,Pitch Black (2000),Horror Sci-Fi Thriller,David Twohy None Vin Diesel Claudia Black Keit...


In [36]:
movies_with_movie_index.drop(['title', 'popularity', 'tmdb_avgRating', 'release_year_int', 'document', 'Writers'], axis=1, inplace=True)

In [37]:
movies_with_movie_index.head()

,primaryTitle,Directors,tmdb_id,company_name,genres
0,Hollow Man,Paul Verhoeven,9383,Columbia Pictures Red Wagon Entertainment Glob...,Horror Sci-Fi Thriller
1,Nurse Betty,Neil LaBute None,10480,IMF Internationale Medien und Film GmbH & Co. ...,Comedy Crime Drama Romance Thriller
2,Supernova,Walter Hill,10384,Hammerhead Productions Screenland Pictures Uni...,Adventure Sci-Fi Thriller
3,Boiler Room,Ben Younger None,14181,New Line Cinema,Crime Drama Thriller
4,Pitch Black,David Twohy None,2787,Interscope Communications,Horror Sci-Fi Thriller


In [12]:
movies_with_tag.drop(['index', 'title', 'popularity', 'tmdb_avgRating', 'release_year_int', 'document'], axis=1, inplace=True)

In [13]:
movies_with_tag.head()

,primaryTitle,Directors,Writers,tmdb_id,company_name,movieId,genres
0,Kate & Leopold,James Mangold None,Steven Rogers,11232,Konrad Pictures Miramax,80,Comedy Romance
1,The Other Side of the Wind,Orson Welles None,Dax Phelan,299782,SACI Les Films de l'Astrophore,13231,Comedy Drama
2,Crime and Punishment,Menahem Golan None,Fyodor Dostoevsky Ivan Mendzheritskiy,109809,NaN,21528,Drama
3,Master i Margarita,Yuriy Kara,Mikhail A. Bulgakov Felix Kroll,63163,Tvorcheskaya Assotsiatsiya Mezhdunarodnykh Pro...,14074,Drama Fantasy Mystery Romance
4,The Fantasticks,Michael Ritchie,Harvey Schmidt Tom Jones,62127,Sullivan Street Productions Michael Ritchie Pr...,7948,Musical


In [14]:
ratings.drop(['timestamp'], axis=1, inplace=True)

In [15]:
with open('./pkl/movie_to_index.pkl', 'rb') as movie_mapping:
    movie_to_index = pickle.load(movie_mapping)
with open('./pkl/user_to_index.pkl', 'rb') as user_mapping:
    user_to_index = pickle.load(user_mapping)
ratings.movieId = ratings.movieId.apply(lambda x: movie_to_index[x])
ratings.userId = ratings.userId.apply(lambda x: user_to_index[x])

In [16]:
ratings.head()

,userId,movieId,rating
0,0,0,2.0
1,0,1,3.5
2,1,2,3.5
3,1,3,4.5
4,1,4,3.5


In [17]:
popularity = pd.DataFrame(ratings[['userId', 'movieId']].groupby(['movieId']).agg(['count']))
popularity.reset_index(inplace=True)
popularity.columns = ['movieId', 'ratings_count']
popularity.sort_values('ratings_count', ascending=False, inplace=True)

In [24]:
popularity.head()

,movieId,ratings_count
81,81,61883
157,157,57378
116,116,56696
12,12,48666
53,53,46826


In [19]:
average_ratings = pd.DataFrame(ratings[['rating', 'movieId']].groupby(['movieId']).agg(['mean']))
average_ratings.reset_index(inplace=True)
average_ratings.columns = ['movieId', 'avg_rating']

In [23]:
average_ratings.head()

,movieId,avg_rating
0,0,2.546134
1,1,3.165050
2,2,2.328496
3,3,3.498866
4,4,3.443778


In [38]:
movies_index = movies_with_movie_index.merge(popularity, left_index=True, right_on='movieId')

In [40]:
movies_index = movies_index.merge(average_ratings, left_index=True, right_on='movieId')

In [45]:
movies_index.to_csv('data/all_movies_movie_index_rating.csv', index=False)

In [27]:
movies = movies_with_tag.merge(popularity, on='movieId')

In [29]:
movies = movies.merge(average_ratings, on='movieId')

In [32]:
movies.head()

,primaryTitle,Directors,Writers,tmdb_id,company_name,movieId,genres,ratings_count,avg_rating
0,Kate & Leopold,James Mangold None,Steven Rogers,11232,Konrad Pictures Miramax,80,Comedy Romance,4178,3.155936
1,The Other Side of the Wind,Orson Welles None,Dax Phelan,299782,SACI Les Films de l'Astrophore,13231,Comedy Drama,9,2.444444
2,Crime and Punishment,Menahem Golan None,Fyodor Dostoevsky Ivan Mendzheritskiy,109809,NaN,21528,Drama,1,1.000000
3,Master i Margarita,Yuriy Kara,Mikhail A. Bulgakov Felix Kroll,63163,Tvorcheskaya Assotsiatsiya Mezhdunarodnykh Pro...,14074,Drama Fantasy Mystery Romance,10,3.200000
4,The Fantasticks,Michael Ritchie,Harvey Schmidt Tom Jones,62127,Sullivan Street Productions Michael Ritchie Pr...,7948,Musical,95,2.921053


In [43]:
movies[movies['movieId'] == 21638]

,primaryTitle,Directors,Writers,tmdb_id,company_name,movieId,genres,ratings_count,avg_rating
16655,Hotline,Tony Shaff,NaN,267355,NaN,21638,Documentary,1,3.5


In [34]:
movies.to_csv('data/all_movies_with_rating.csv', index=False)